# GReaT colab example

**Please make sure you select a runtime with GPU**

*(Runtime-> Change Runtime-> Hardware accelerator is set to GPU)*

In [1]:
%%capture
!pip install be-great
!pip install openpyxl

In [2]:
!pip install xlrd

In [3]:
from be_great import GReaT
import pandas as pd
import torch
import xlrd
from openpyxl import load_workbook

In [4]:
# Create a function to get the cell color
def get_cell_color(cell, workbook):
    xfx = cell.xf_index
    xf = workbook.xf_list[xfx]
    bg_color_index = xf.background.pattern_colour_index
    return workbook.colour_map[bg_color_index]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
## Reading in Schema file - Medical Claims
file_path = '/content/drive/MyDrive/Capgemini-Practicum/Dataset/Data Dictionary Medical Claims.xlsx'
df_medical_schema = pd.read_excel(file_path, skiprows=1)

In [7]:
df_medical_schema.shape

(84, 2)

In [8]:
column_indices_to_keep=[3,13,16,18,19,20,23,24,25,31,32,33,36,49,52,67,68,75,76,80]
##adjusting for indices
column_indices_to_keep=[i-3 for i in column_indices_to_keep]

In [9]:
column_indices_to_keep

[0, 10, 13, 15, 16, 17, 20, 21, 22, 28, 29, 30, 33, 46, 49, 64, 65, 72, 73, 77]

In [10]:
df_medical_schema=df_medical_schema.iloc[column_indices_to_keep].reset_index(drop=True)

In [11]:
dtypes = {'_c21': 'str', '_c61': 'str'}
df_medical_data = pd.read_csv("drive/MyDrive/Capgemini-Practicum/Dataset/medical_claims.csv.gz", usecols=column_indices_to_keep,dtype=dtypes,nrows=100000)
print(df_medical_data.shape)

(100000, 20)


In [12]:
df_medical_data.columns=list(df_medical_schema['Field'])

In [13]:
df_medical_data

,Member Life ID,Gender Code,Subscriber Zip Code,Claim Type Code,Claim Number,Line #,Network Indicator,Bill Type,Internal Provider Number,Place of Service Code,Type of Service Code,Current Procedural Terminology,Revenue Codes,Primary Diagnosis Code-ICD10,ICD10 Surgical Procedure Code 1,Line Service From Date,Line Service thru Date,# of Services,Billed Amount,Paid Amount
0,3269408,M,20769,P,831210364500,7,I,NaN,S1690000,11,7D0,97014,*,M9904,*,2018-10-23,2018-10-23,1,28.00,0.00
1,3269408,M,20769,P,835410338700,1,I,NaN,S1690000,11,600,98941,*,S335XXA,*,2018-11-13,2018-11-13,1,50.00,5.23
2,3269408,M,20769,P,835410338700,3,I,NaN,S1690000,11,7D0,97140,*,S335XXA,*,2018-11-13,2018-11-13,1,45.00,18.14
3,3269408,M,20769,P,835410860300,2,I,NaN,S1690000,11,7D0,97012,*,S335XXA,*,2018-12-04,2018-12-04,1,55.00,9.84
4,3269408,M,20769,P,835410860300,4,I,NaN,S1690000,11,7D0,97014,*,S335XXA,*,2018-12-04,2018-12-04,1,28.00,9.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,25791520,M,61073,I,020183090122101,4,I,131,999Y2AA1068ABD,22,200,J7120,0636,Z1211,0DJD8ZZ,2018-10-30,2018-10-30,-1,-209.25,-88.52
99996,25791520,M,61073,I,020183090122101,6,I,131,999Y2AA1068ABD,22,200,*,0250,Z1211,0DJD8ZZ,2018-10-30,2018-10-30,-1,-69.40,-29.36
99997,25791520,M,61073,I,020183090122102,1,I,137,999Y2AA1068ABD,22,200,45380,0750,Z1211,0DJD8ZZ,2018-10-30,2018-10-30,1,2195.00,928.49
99998,25791520,M,61073,I,020183090122102,2,I,137,999Y2AA1068ABD,22,200,*,0370,Z1211,0DJD8ZZ,2018-10-30,2018-10-30,30,433.50,183.38


In [14]:
# df_medical_data[df_medical_data['Revenue Codes']=='*'].shape
df_medical_data[df_medical_data['ICD10 Surgical Procedure Code 1']=='*'].shape

(95852, 20)

In [15]:
model = GReaT(llm='distilgpt2', batch_size=16, epochs=1, save_steps=400000)

In [16]:
torch.cuda.empty_cache()

In [17]:
model.fit(df_medical_data)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.224300
1000,0.974700
1500,0.931600
2000,0.911000
2500,0.894700
3000,0.881700
3500,0.872300
4000,0.862000
4500,0.856500
5000,0.852600


  0%|          | 0/100 [00:21<?, ?it/s]

An error has occurred: Breaking the generation loop!
To address this issue, consider fine-tuning the GReaT model for an longer period. This can be achieved by increasing the number of epochs.
Alternatively, you might consider increasing the max_length parameter within the sample function. For example: model.sample(n_samples=10, max_length=2000)
If the problem persists despite these adjustments, feel free to raise an issue on our GitHub page at: https://github.com/kathrinse/be_great/issues


In [18]:
# Save the entire model
torch.save(model, 'model.pth')

In [23]:
synthetic_data = model.sample(n_samples=100,max_length=2000)

154it [00:06, 24.23it/s]


In [24]:
synthetic_data

,Member Life ID,Gender Code,Subscriber Zip Code,Claim Type Code,Claim Number,Line #,Network Indicator,Bill Type,Internal Provider Number,Place of Service Code,Type of Service Code,Current Procedural Terminology,Revenue Codes,Primary Diagnosis Code-ICD10,ICD10 Surgical Procedure Code 1,Line Service From Date,Line Service thru Date,# of Services,Billed Amount,Paid Amount
0,33782189.0,F,21613.0,I,0201830505034300,1.0,I,131,580FCMC6410001,22.0,6A0,99213,0307,R0602,*,2018-10-18,2018-10-18,1.0,4.56,0.00
1,1825194.0,M,21093.0,P,026172740278800,2.0,I,811,580PD745410001,81.0,8A0,83520,*,Z01419,*,2017-06-26,2017-06-26,1.0,118.06,58.21
2,279860.0,F,20036.0,P,830417686800,1.0,I,None,D6130001,11.0,600,99214,*,M25562,*,2018-07-28,2018-07-28,1.0,250.00,0.00
3,244837.0,F,21774.0,P,824216861500,1.0,I,None,580PW423419998,11.0,600,99214,*,Z23,*,2018-11-25,2018-11-25,1.0,35.00,18.19
4,7183382.0,M,20720.0,P,826010834100,1.0,I,None,D7450001,11.0,6S0,99203,*,M545,*,2018-01-11,2018-01-11,1.0,150.00,99.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,35081577.0,F,21163.0,P,811611134800,1.0,I,None,999Y2A9F8D6C9,81.0,800,82370,*,F419,*,2018-04-13,2018-04-13,1.0,50.00,23.61
96,32799325.0,M,21012.0,P,830516337900,1.0,I,None,21790000,11.0,600,99213,*,R8283,*,2018-02-21,2018-02-21,1.0,40.00,0.00
97,33989958.0,M,22304.0,P,026181954641400,1.0,I,111,580PW274419998,11.0,600,90471,*,Z01419,*,2018-03-06,2018-03-06,1.0,66.00,3.57
98,34234515.0,M,20659.0,P,026180793700,3.0,I,None,999Y2A721D25E,11.0,9A0,E1390,*,M542,*,2018-03-16,2018-03-16,1.0,25.00,0.00
